<a href="https://colab.research.google.com/github/AbdoolAK/MRI/blob/Odysseas/trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook adds zero-padding to the mri scans of different institutes
* This is necessary to bring the scans in the same dimensions
* It has already been run, and the padded images have been saved, so no need to run again

Click on the cell above to run it in Google Colab

In [ ]:
# Check Python version
!python --version

Python 3.8.16


In [2]:
# Colab doesn't automatically come with a few packages, we install them here
!pip install simpleitk
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 14.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 37.3 MB/s eta 0:00:00


In [3]:
%load_ext tensorboard
from pathlib import Path
import torch
import os

import torchvision.transforms as transforms
import SimpleITK as sitk

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
from google.colab import drive

drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


Now that we have a link to our Google Drive, download the WMH data, and put it in there.
Remember where you stored it, we'll need to access it later!

In [5]:
# Check to see if the data exists (replace the path with your own)
base_path = Path("/content/MyDrive/MyDrive/")
dataset_path = base_path / "AI_for_medical_imaging/Images"
tensorboard_path = dataset_path / "Tensorboard_logs"

if not tensorboard_path.exists():
    tensorboard_path.mkdir(parents=True)

!ls "$dataset_path"

Amsterdam  images_pad  Singapore  Tensorboard_logs  Utrecht


In [6]:
# Function that returns images as tensor 
def read_image(path):
    img = sitk.ReadImage(path)
    img_as_numpy = sitk.GetArrayFromImage(img).astype('float') # the default type is uint16, which trips up PyTorch so we convert to float
    img_as_tensor = torch.from_numpy(img_as_numpy)
    return img_as_tensor

In [7]:
def save_image(image, path):
    new_image = sitk.GetImageFromArray(image.squeeze())
    sitk.WriteImage(new_image, path, True)

In [ ]:
def pad_images(institute: str):
    base_path = Path("/content/MyDrive/MyDrive/")
    dataset_path = base_path / "AI_for_medical_imaging/Images"
    imgs = os.listdir(dataset_path)
    institute_path = dataset_path / institute / institute
    
    for file in os.listdir(institute_path):
        
        if str(file).isdigit():
            gt_path = institute_path / Path(file)
            for img in os.listdir(gt_path):
              if img!= "pre":
                old = read_image(gt_path / img)
                h, w = old.shape[1], old.shape[2]
                transform = transforms.Pad((int(abs(256-w)/2), int(abs(h-256)/2), int(abs(256-w)/2), int(abs(h-256)/2)))
                new = transform(old)
                new[new != new] = 0      # to eliminate NaN values
                new_path_pad_gt = '/content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/' + institute + '/' + str(file)

                file_name, ext = os.path.splitext(img)
                file_name, ext = os.path.splitext(file_name)

                print('Saving GT ... ' + new_path_pad_gt + '/PAD_' + file_name + '.nii.gz')
                save_image(new, str(new_path_pad_gt + '/PAD_' + file_name + '.nii.gz'))

              patient_path = institute_path / Path(file) / "pre"

            for img in os.listdir(patient_path):
                old = read_image(patient_path / img)
                h, w = old.shape[1], old.shape[2]
                transform = transforms.Pad((int(abs(256-w)/2), int(abs(h-256)/2), int(abs(256-w)/2), int(abs(h-256)/2)))
                new = transform(old)
                new[new != new] = 0   # to eliminate NaN values

                new_path_pad = '/content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/' + institute + '/' + file

                if not os.path.exists(new_path_pad):
                   os.makedirs(new_path_pad)

                file_name, ext = os.path.splitext(img)
                file_name, ext = os.path.splitext(file_name)

                print('Saving IMG... ' + new_path_pad + '/PAD_' + file_name + '.nii.gz')
                save_image(new, str(new_path_pad + '/PAD_' + file_name + '.nii.gz'))
                
                del old, new

In [ ]:
pad_images("Amsterdam")

Saving GT ... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/104/PAD_wmh.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/104/PAD_FLAIR.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/104/PAD_T1.nii.gz
Saving GT ... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/100/PAD_wmh.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/100/PAD_T1.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/100/PAD_FLAIR.nii.gz
Saving GT ... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/101/PAD_wmh.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/101/PAD_FLAIR.nii.gz
Saving IMG... /content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/Amsterdam/101/PAD_T1.nii.gz
Saving GT ... /c

In [10]:
#make sure the padding was correct and that the images downloaded ok
institute=('Amsterdam')

path_images_not_pad=('/content/MyDrive/MyDrive/AI_for_medical_imaging/Images/' + institute + '/' + institute)

path_images_pad=('/content/MyDrive/MyDrive/AI_for_medical_imaging/Images/images_pad/' + institute)

for file in os.listdir(path_images_not_pad):
        print('\n', file)
        if str(file).isdigit():
            gt_path = path_images_not_pad / Path(file)
            for img in os.listdir(gt_path):
              if img!= "pre":
                 print(img)
                 image = read_image(gt_path / img)        # do the original amsterdam images have NaN values?
                 result=torch.isnan(image).any()
                 print('IMAGE NOT CHANGED: ', result)
                 print(image.shape)

            patient_path = path_images_not_pad / Path(file) / "pre"
            for img in os.listdir(patient_path):
                print(img)
                image = read_image(patient_path / img)
                result=torch.isnan(image).any()          # do the padded amsterdam images have NaN values?
                print('IMAGE NOT CHANGED: ', result)
                print(image.shape)

for file in os.listdir(path_images_pad):
        print('\n', file)
        if str(file).isdigit():
            patient_path = path_images_pad / Path(file)
            for img in os.listdir(patient_path):
                image = read_image(patient_path / img)
                result=torch.isnan(image).any()
                print('IMAGE CHANGED: ', result)
                print(image.shape)


 104
wmh.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
FLAIR.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
T1.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])

 100
wmh.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
T1.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
FLAIR.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])

 101
wmh.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
FLAIR.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
T1.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])

 102
wmh.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
FLAIR.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
T1.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])

 103
wmh.nii.gz
IMAGE NOT CHANGED:  tensor(False)
torch.Size([83, 256, 132])
FLAIR.nii.gz
IMAGE NOT CHANGED